In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext tensorboard
import datetime


#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

In [2]:
!rm -rf ./logs/GRU #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        training_data_y = np.load(f)
        test_y = np.load(f)
        
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x) 

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [5]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

Epoch 1/3
  2/221 [..............................] - ETA: 28:53 - loss: 0.7359 - accuracy: 0.6875 - auc: 0.6682 - precision: 0.6129 - recall: 0.7037 - false_negatives: 8.0000 - false_positives: 12.0000 - true_negatives: 25.0000 - true_positives: 19.0000

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_18737/1449073585.py", line 19, in <module>
    RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1189, in fit
    callbacks.on_train_batch_end(end_step, logs)
  File "/opt/conda/lib/python3.7/site-packages/keras/callbacks.py", line 435, in on_train_batch_end
    self._call_batch_hook(ModeKeys.TRAIN, 'end', batch, logs=logs)
  File "/opt/conda/lib/python3.7/site-packages/keras/callbacks.py", line 295, in _call_batch_hook
    self._call_batch_end_hook(mode, batch, logs)
  File "/opt/conda/lib/python3.7/site-packages/keras/callbacks.py", line 315, in _call_batch_end_hook
    self._call_batch_

TypeError: object of type 'NoneType' has no len()

In [ ]:
%tensorboard --logdir logs/RNN/fit